# Evaluator trainer

In [ ]:
import time
import random
import json
import tensorflow as tf
import numpy as np
import evaluator_model
import cv2
import glob
###########################START DATA LOADER#############################
batch_size = 128
L_balloon = len(glob.glob('./dataset/balloon/*.png'))
L_bulb = len(glob.glob('./dataset/bulb/*.png'))
L_ice  =len(glob.glob('./dataset/ice/*.png'))
L_mic = len(glob.glob('./dataset/mic/*.png'))

balloon_paths = glob.glob('./dataset/balloon/*.png')
bulb_paths = glob.glob('./dataset/bulb/*.png')
ice_paths = glob.glob('./dataset/ice/*.png')
mic_paths = glob.glob('./dataset/mic/*.png')

L_4 = [L_balloon,L_bulb,L_ice,L_mic]
P_4 = [balloon_paths,bulb_paths,ice_paths,mic_paths]
def gen_batch(batch_size = 64):
    x = np.zeros((batch_size,64,64))
    y = np.zeros(batch_size,)
    genre = np.random.randint(0,4,batch_size)
    for gi,g in enumerate(genre):
        which = np.random.randint(0,L_4[g])
        x[gi,:,:] = cv2.imread(P_4[g][which], cv2.IMREAD_GRAYSCALE)/255
        y[gi] = int(g)
    return x,y
###########################END DATA LOADER#############################


EVA = evaluator_model.Evaluate_model(image_size = 64)

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=EVA.labels, logits=EVA.predictions)
loss = tf.reduce_mean(cross_entropy, name="loss")
optimizer = tf.train.AdamOptimizer(0.0001, 0.9)
training_op = optimizer.minimize(loss,name = "training_op")

init = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep=5)


config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    init.run()
    try:
        saver.restore(sess, "./fuji_ckpt/model_8000.ckpt")
        print('model loaded')
    except:
        print('model not found')
    correct = 0
    print_n  = 100
    for i in range(100000):
        X, y = gen_batch(batch_size)

        _,train_loss,pred = sess.run([training_op,loss,EVA.predictions], feed_dict={ EVA.input_x: X, EVA.labels: y})
        print(i,train_loss)
        y  = y.astype(int)
        pred = np.argmax(pred,axis = 1)
        correct += np.sum(np.equal(y, pred))
        if i %print_n == 0:

            print(pred[:20])
            print(y[:20])
            print('acc:',correct/(print_n*batch_size))
            correct = 0

        if i %1000 == 0:
            saver.save(sess,'./fuji_ckpt/model_'+str(i)+'.ckpt')
            print('model save')



# RUSH algorithm

In [ ]:
import numpy as np
from utils import *
class SRNN_Model:
    def __init__(self, hps, model_name='team-2-microphone-bulb'):
        which = model_name.split('-')[2]
        if which == 'ice':
            self.buffer = np.array([
           [ 0.33723726,  0.9 ,  1.        ,  0.        ,  0.        ],
           [ 0.33801074,  0.9,  1.        ,  0.        ,  0.        ],
           [ 0.33728634,  0.9,  1.        ,  0.        ,  0.        ],
           [ 0.20952217,  0.46887963,  1.        ,  0.        ,  0.        ],
           [ 0.33425487, -0.68217094,  1.        ,  0.        ,  0.        ],
           [ 0.54911187, -0.96608985,  1.        ,  0.        ,  0.        ],
           [ 0.1767253 , -0.53924123,  1.        ,  0.        ,  0.        ],
           [ 0.038424  , -0.92267113,  1.        ,  0.        ,  0.        ],
           [-0.43504955, -0.19516976,  1.        ,  0.        ,  0.        ],
           [-1.0035214, -0.0194721 ,  1.        ,  0.        ,  0.        ],
           [-0.73898667,  0.05936348,  1.        ,  0.        ,  0.        ],
           [-0.28474408, -0.23035759,  1.        ,  0.        ,  0.        ],
           [ 0.06897274, -0.39256686,  1.        ,  0.        ,  0.        ],
           [ 0.56570638, -0.39613263,  1.        ,  0.        ,  0.        ],
           [ 0.53795129, -0.22199969,  1.        ,  0.        ,  0.        ],
           [ 0.53795129, 0.12199969,  1.        ,  0.        ,  0.        ],
           [ 0.700000 ,  0.4,  1.        ,  0.        ,  0.        ],
           [ 0.15061277,  0.6784492 ,  0.        ,  1.        ,  0.        ],
           [ -1.0000000,  -1.000000 ,  1.        ,  0.        ,  0.        ],
           [ -0.45061277,  -0.1784492 ,  1.        ,  0.        ,  0.        ],
           [ -0.25061277,  -0.2784492 ,  1.        ,  0.        ,  0.        ],
           [ -0.15061277,  -0.4784492 ,  1.        ,  0.        ,  0.        ],
           [ 0.25061277,  -0.2784492 ,  1.        ,  0.        ,  0.        ],
           [ 0.45061277,  -0.1784492 ,  1.        ,  0.        ,  0.        ],
           [ 0.55061277,  -0.0000000 ,  1.        ,  0.        ,  0.        ],
           [ 0.45061277,  0.1784492 ,  1.        ,  0.        ,  0.        ],
           [ 0.25061277,  0.2784492 ,  1.        ,  0.        ,  0.        ],
           [ -0.15061277,  0.4784492 ,  1.        ,  0.        ,  0.        ],
           [ -0.25061277,  0.2784492 ,  1.        ,  0.        ,  0.        ],
           [ -0.445061277,  0.1784492 ,  0.        ,  0.        ,  1.        ],
            ])
        elif which == 'microphone':
            self.buffer = np.array([[ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ],
       [-0.2024052,  1.0,  1.        ,  0.        ,  0.        ],
       [-0.20789843,  1.0,  1.        ,  0.        ,  0.        ],
       [-0.20789843,  1.0,  1.        ,  0.        ,  0.        ],
       [-0.20789843,  1.0,  1.        ,  0.        ,  0.        ],
       [ 0.3,  0.0,  1.        ,  0.        ,  0.        ],
       [ 0.17849151, -0.84414113,  1.        ,  0.        ,  0.        ],
       [ 0.17849151, -0.84414113,  1.        ,  0.        ,  0.        ],
       [ 0.17849151, -0.84414113,  1.        ,  0.        ,  0.        ],
       [ 0.17849151, -0.84414113,  1.        ,  0.        ,  0.        ],             
       [ 0.07664307, -0.47148524,  1.        ,  0.        ,  0.        ],
       [ 0.47441457, -0.3003108,  1.        ,  0.        ,  0.        ],
       [ 0.20264079, -0.39200626,  1.        ,  0.        ,  0.        ],
       [ -0.20264079, -0.39200626,  1.        ,  0.        ,  0.        ],
        [ -0.80264079, -0.39200626,  1.        ,  0.        ,  0.        ], 
        [ -0.5264079, 0.39200626,  1.        ,  0.        ,  0.        ],
       [ -0.20264079, 0.39200626,  1.        ,  0.        ,  0.        ],
       [ 0.20264079, 0.39200626,  1.        ,  0.        ,  0.        ],
       [ 0.5264079, 0.39200626,  1.        ,  0.        ,  0.        ],
       [ 0.5264079, 0.0200626,  0.        ,  1.        ,  0.        ],
       [ -0.9, 0.7    ,  0. ,      1.   ,  0.        ],
       [ 0.0, 1.0     ,  0.,      1.  ,  0.        ],
       [ 0.0, 1.0     ,  0.,      1. ,  0.        ],
       [ 0.0, 1.0    ,  1. ,      0.  ,  0.        ],
       [ -1.0, 1.0    ,  1. ,      0.  ,  0.        ],
       [ 0.5, 1.0  , 1. ,      0.  ,  0.        ],
       [ -1.0, 1.0    ,  1. ,      0.  ,  0.        ],
       [ 0.5, 1.0  , 0. ,      0.  ,  1.        ],
        ])
        elif which == 'balloon':
            self.buffer = np.array([
        [-1.0,  0.0930948 ,  1.        ,  0.        ,  0.        ],
        [ -0.58941234,  1.0,  1.        ,  0.        ,  0.        ],         
        [ 1.0, -0.13087307,  1.        ,  0.        ,  0.        ],
        [ 1.0, -0.13087307,  1.        ,  0.        ,  0.        ],
        [ 1.0, -0.13087307,  1.        ,  0.        ,  0.        ],
        [-0.57323594, -1.0,  1.        ,  0.        ,  0.        ],
        [-1.0,  0.0930948 ,  1.        ,  0.        ,  0.        ],
        [ 0.3812114, -1.0,  1.        ,  0.        ,  0.        ],
        [-0.57068183, -0.3,  1.        ,  0.        ,  0.        ],
        [-0.37068183, -0.3,  1.        ,  0.        ,  0.        ],
        [-0.32875838, -0.48067306,  1.        ,  0.        ,  0.        ],
        [-0.03832597, -0.8,  1.        ,  0.        ,  0.        ],
        [ 0.62901495, -0.33775914,  1.        ,  0.        ,  0.        ],
        [1.0, -0.12718418,  1.        ,  0.        ,  0.        ],
        [ 0.8,  0.3,  1.        ,  0.        ,  0.        ],
        [ 0.8,  0.5,  1.        ,  0.        ,  0.        ],
        [-0.07574196,  0.51950459,  1.        ,  0.        ,  0.        ],
        [-0.56015114,  0.86562866,  1.        ,  0.        ,  0.        ],
        [-0.63389954,  0.14718226,  1.        ,  0.        ,  0.        ],
        [-0.0,  0.8718226,  0.        ,  1.        ,  0.        ],
        [-0.4,  -0.8718226,  1.        ,  0.        ,  0.        ],
        [-0.0,  0.8718226,  0.        ,  1.        ,  0.        ],
        [-0.4,  -0.8718226,  1.        ,  0.        ,  0.        ],
        [-0.0,  0.8718226,  0.        ,  0.        ,  1.        ],
        ])
        elif which == 'bulb':
            self.buffer = np.array([
       [ 0.17413329,  0.78435835,  1.        ,  0.        ,  0.        ],
       [ 1.0, -0.11103143,  1.        ,  0.        ,  0.        ],
       [ -1.0, -0.11103143,  1.        ,  0.        ,  0.        ],  
       [ 1.0, -0.1103143,  1.        ,  0.        ,  0.        ],  
       [-1.0, -0.11103143,  1.        ,  0.        ,  0.        ],  
       [ 1.0, -0.1103143,  1.        ,  0.        ,  0.        ],  
       [-0.11992865, -0.304639 ,  1.        ,  0.        ,  0.        ],
       [ 0.35024508, -0.40397106,  1.        ,  0.        ,  0.        ],
       [ 0.29604211, -0.53311938,  1.        ,  0.        ,  0.        ],
       [ 0.05978016, -0.48035347,  1.        ,  0.        ,  0.        ],
       [-0.15608428, -0.56221507,  1.        ,  0.        ,  0.        ],
       [-0.20017952, -0.23626968,  1.        ,  0.        ,  0.        ],
       [-0.45796524, -0.32520056,  1.        ,  0.        ,  0.        ],
       [-0.54606161, -0.16006442,  1.        ,  0.        ,  0.        ],
       [-0.44380822,  0.02632717,  1.        ,  0.        ,  0.        ],
       [-0.40674807,  0.2996969 ,  1.        ,  0.        ,  0.        ],
       [-0.37408802,  0.53965401,  1.        ,  0.        ,  0.        ],
       [-0.1467784 ,  0.51184686,  1.        ,  0.        ,  0.        ],
       [ 0.05092647,  0.6054288 ,  1.        ,  0.        ,  0.        ],
       [ 0.21702092,  0.41537722,  1.        ,  0.        ,  0.        ],
       [ 0.41630971,  0.35970099,  1.        ,  0.        ,  0.        ],
       [ 0.47605845,  0.15727269,  1.        ,  0.        ,  0.        ],
       [ 0.52474531, -0.09532492,  1.        ,  0.        ,  0.        ],
       [ 0.65024508, -0.50397106,  1.        ,  0.        ,  0.        ],
       [ 0.29604211, -0.53311938,  1.        ,  0.        ,  0.        ],
       [ 0.05978016, -0.48035347,  1.        ,  0.        ,  0.        ],
       [-0.15608428, -0.56221507,  1.        ,  0.        ,  0.        ],
       [-0.20017952, -0.23626968,  1.        ,  0.        ,  0.        ],
       [-0.45796524, -0.32520056,  1.        ,  0.        ,  0.        ],
       [-0.54606161, -0.16006442,  1.        ,  0.        ,  0.        ],
       [-0.44380822,  0.02632717,  1.        ,  0.        ,  0.        ],
       [-0.40674807,  0.2996969 ,  1.        ,  0.        ,  0.        ],
       [-0.37408802,  0.53965401,  1.        ,  0.        ,  0.        ],
       [-0.1467784 ,  0.51184686,  1.        ,  0.        ,  0.        ],
       [ 0.05092647,  0.6054288 ,  1.        ,  0.        ,  0.        ],
       [ 0.21702092,  0.41537722,  1.        ,  0.        ,  0.        ],
       [ 0.41630971,  0.35970099,  1.        ,  0.        ,  0.        ],
       [ 0.47605845,  0.15727269,  1.        ,  0.        ,  0.        ],
       [ 0.52474531, -0.09532492,  0.        ,  0.        ,  1.        ],
        ])
        self.out_ptr = 0
        self.prev_pos = 0
        self.tie_mode = False
    def generate_stroke(self, sess, prev_sketch):

        if len(prev_sketch) != 1:
            # if player 2 act first (slopy check)
            if prev_sketch[1,0] != self.buffer[0,0] and len(prev_sketch)<=11:

                self.new_draw_init()
                out =  self.draw(10)
            else:
                p2_draw = prev_sketch[self.prev_pos:]
                # if len(p2_draw) !=0:
                try:
                  p2_back = self.go_back(p2_draw)
                  out = self.draw(10-len(p2_back))
                  out = np.concatenate((p2_back,out),axis=0)
                except:
                  out = self.draw(10)
        # if player 1 act first
        else:
            self.new_draw_init()
            out =  self.draw(10)
        
        self.prev_pos = len(prev_sketch) + len(out)

        # if(self.out_ptr >= len(self.buffer)):

        #     self.tie_mode = True

        # if self.tie_mode:
        #     return np.tile([1,1,0,1,0], (10, 1))


        return out
        
    def new_draw_init(self):
        self.out_ptr = 0
        self.tie_mode = False
    def draw(self,n):
        try:
            out = self.buffer[self.out_ptr:self.out_ptr+n,:]
        except:
            n = min(len(self.buffer),self.out_ptr)
            out = self.buffer[n:,:]
            
        self.out_ptr+=n
        return out
    def load_model(self, sess,checkpoint_dir):
        pass
    def go_back(self,history):
        # assert(len(history) != 0)

        x,y = np.sum(history,axis=0)[:2]
        n = int(np.ceil(max(abs(x),abs(y))))
        # if n==0:
        #   return np.array([[-x,-y,1,0,0]])
        lst = np.zeros((n,5))
        x_ = x/n
        y_ = y/n

        lst[:,0] = -x_
        lst[:,1] = -y_
        lst[:,3] = 1
        lst[-1,3] = 0
        lst[-1,2] = 1
        return lst
def get_default_hparams():
    return []